In [ ]:
import doctest
from itertools import permutations
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')


In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

FSA = []
for post in soup.find_all('td'):
    code = post.text
    if code == '':
        break
    else:
        last_chr = int(len(code))
        last_chr = last_chr - 1
        if code[last_chr] == '\n':
            code = code[:-1]
        FSA.append(code)
post_dict = {}
postcode = []
burrough = []
neighbourhood = []
while len(FSA) > 2:
    postcode.append(FSA[0])
    burrough.append(FSA[1])
    neighbourhood.append(FSA[2])
    del FSA[0:3]

post_dic = {'Postal Code':postcode, 'Burrough':burrough, 'Neighborhood':neighbourhood}

df_post = pd.DataFrame(post_dic)

df_post = df_post[df_post.Burrough != 'Not assigned']

df_post = df_post.groupby(['Postal Code', 'Burrough'])['Neighborhood'].apply(', '.join).reset_index()

df_post.loc[df_post.Neighborhood == 'Not assigned', 'Neighborhood'] = df_post.Burrough

#print(df_post)

df_post.shape

In [ ]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_new = pd.merge(df_post, df_geo, on='Postal Code')
df_new.head()

In [ ]:
address = 'Toronto, CN'

geolocater = Nominatim()
location = geolocater.geocode(address)
latitude = location.longitude
longitude = location.longitude
print('The geographical coordinate of Toronto, Cananda are {}, {}'.format(latitude, longitude))

In [ ]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng, burough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Burrough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, burough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_Toronto)
    
map_Toronto

In [ ]:
Toronto_west = df_new[df_new['Burrough'] == 'West Toronto'].reset_index(drop=True)
Toronto_east = df_new[df_new['Burrough'] == 'East Toronto'].reset_index(drop=True)
Toronto_down = df_new[df_new['Burrough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_central = df_new[df_new['Burrough'] == 'Central Toronto'].reset_index(drop=True)
Toro = pd.concat([Toronto_west, Toronto_east,Toronto_down,Toronto_central])
Toro.reset_index(drop=True, inplace=True)

Toro

In [ ]:
### TORONTO NEIGHBORHOODS -- WEST, EAST, DOWNTOWN and CENTRAL

map_Toronto_neigh = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng, burough, neighborhood in zip(Toro['Latitude'], Toro['Longitude'], Toro['Burrough'], Toro['Neighborhood']):
    label = '{}, {}'.format(neighborhood, burough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_Toronto_neigh)
    
map_Toronto_neigh


In [ ]:
CLIENT_ID = '3M5MD4WTFDBCTGKO2CFVH120NXIJ1R43IRBJTVU0BGGIMSUP'
CLIENT_SECRET = 'AJRZ10JWEYOTWU44WM3JC04XJKQN2Y3XB2T5DHS5JFUMOYGX'
VERSION = '20180605'

In [ ]:
radius = 500
LIMIT = 100
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID,
#     CLIENT_SECRET,
#     VERSION,
#     neighborhood_latitude,
#     neighborhood_longitude,
#     radius,
#     LIMIT)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(
            name, 
            lat,
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                            'Neighborhood Latitude', 
                            'Neighborhood Longitude', 
                            'Venue', 
                            'Venue Latitude', 
                            'Venue Longitude', 
                            'Venue Category']
    return(nearby_venues)


In [ ]:
#print(Toro['Neighborhood'][1])
toro_venues = getNearbyVenues(names=Toro['Neighborhood'], latitudes=Toro['Latitude'], longitudes=Toro['Longitude'])

In [ ]:
print(toro_venues.shape)
print(toro_venues.head())

In [ ]:
toro_venues.groupby("Neighborhood").count()

In [ ]:
print('There are {} unique categories.'.format(len(toro_venues['Venue Category'].unique())))

In [ ]:
toro_venues.columns = ['Neighborhood', 'Neigh_lat', 'Neigh_long', 'Venue', 'Ven_lat', 'Ven_long', 'Ven_cat']
bar_run = toro_venues[toro_venues.Ven_cat = 'Bar']
bar_run = bar_run[bar_run.Neighborhood == 'Studio Distrcit']
li_lat = []
li_lng = []
for lat, lng in zip(bar_run['Ven_lat'], bar_run['Ven_long']):
    li_lat.append(lat)
    li_lng.append(lng)

In [ ]:
### Map of bars for bur_run in Studio District
map_run = folium.Map(location=[latitude, longitude], zoom_start=11)

### Add markers to map
for lat, lng, label in zip(bar_run['Vent_lat'], bar_run['Vent_long'], bar_run['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(bar_run)
    
bar_run

In [ ]:
### define points (lat,lng) for each bar
hold = []
point = []
li_len = len(li_lat)
count = 0
while count < li_len:
    hold.append(li_lat[count])
    hold.append(li_lng[count])
    points.append(hold)
    hold = []
    count = count + 1 
    

In [ ]:
### TSP https://codereview.stackexchange.com/questions/81865/travelling-salesman-using-brute-force-and-heuristics

def distance(point1, point2):
    """
    Returns the Euclidean distance of two points in the Cartesian Plane.

    >>> distance([3,4],[0,0])
    5.0
    >>> distance([3,6],[10,6])
    7.0
    """
    return ((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2) ** 0.5


def total_distance(points):
    """
    Returns the length of the path passing throught
    all the points in the given order.

    >>> total_distance([[1,2],[4,6]])
    5.0
    >>> total_distance([[3,6],[7,6],[12,6]])
    9.0
    """
    return sum([distance(point, points[index + 1]) for index, point in enumerate(points[:-1])])


def travelling_salesman(points, start=None):
    """
    Finds the shortest route to visit all the cities by bruteforce.
    Time complexity is O(N!), so never use on long lists.

    >>> travelling_salesman([[0,0],[10,0],[6,0]])
    ([0, 0], [6, 0], [10, 0])
    >>> travelling_salesman([[0,0],[6,0],[2,3],[3,7],[0.5,9],[3,5],[9,1]])
    ([0, 0], [6, 0], [9, 1], [2, 3], [3, 5], [3, 7], [0.5, 9])
    """
    if start is None:
        start = points[0]
    return min([perm for perm in permutations(points) if perm[0] == start], key=total_distance)


def optimized_travelling_salesman(points, start=None):
    """
    As solving the problem in the brute force way is too slow,
    this function implements a simple heuristic: always
    go to the nearest city.

    Even if this algoritmh is extremely simple, it works pretty well
    giving a solution only about 25% longer than the optimal one (cit. Wikipedia),
    and runs very fast in O(N^2) time complexity.

    >>> optimized_travelling_salesman([[i,j] for i in range(5) for j in range(5)])
    [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [1, 4], [1, 3], [1, 2], [1, 1], [1, 0], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [3, 4], [3, 3], [3, 2], [3, 1], [3, 0], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4]]
    >>> optimized_travelling_salesman([[0,0],[10,0],[6,0]])
    [[0, 0], [6, 0], [10, 0]]
    """
    if start is None:
        start = points[0]
    must_visit = points
    path = [start]
    must_visit.remove(start)
    while must_visit:
        nearest = min(must_visit, key=lambda x: distance(path[-1], x))
        path.append(nearest)
        must_visit.remove(nearest)
    return path


def main():
    doctest.testmod()
    points = [[0, 0], [1, 5.7], [2, 3], [3, 7],
              [0.5, 9], [3, 5], [9, 1], [10, 5]]
    print("""The minimum distance to visit all the following points: {}
starting at {} is {}.

The optimized algoritmh yields a path long {}.""".format(
        tuple(points),
        points[0],
        total_distance(travelling_salesman(points)),
        total_distance(optimized_travelling_salesman(points))))


if __name__ == "__main__":
    main()